In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/iipc-v3/IIPC_Data.csv')
df.head()


,ark_url,title,date,creator,subject,description,item_type,source_url,full_text,cleaned_text
0,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",---\n# **The End of Term Archive: Collaborativ...
1,https://digital.library.unt.edu/ark:/67531/met...,Warcbase: Building a Scalable Web Archiving Pl...,2015-04-28,"Lin, Jimmy",digital preservation; web archiving; historica...,Presentation for the 2015 International Intern...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,Warcbase: Building a Scalable Web Archiving Pl...,---\n## Warcbase: Building a Scalable Web Arch...
2,https://digital.library.unt.edu/ark:/67531/met...,WARC Standard Revision Workshop,2015-04-28,"Oury, Clément",digital preservation; web archiving; web crawl,Presentation for the 2015 International Intern...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,1\nWARC standard revision workshop\nClément Ou...,---\n*1. Workshop Overview**\n**Topic:** WARC ...
3,https://digital.library.unt.edu/ark:/67531/met...,Co-Developing Access to the UK Web Archive,2015-04-27,"Hockx-Yu, Helen",digital preservation; web archiving; development,Presentation for the 2015 International Intern...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,www.bl.uk\n1\nCo-developing access to \nthe UK...,---\n*Title:** Co-developing Access to the UK ...
4,https://digital.library.unt.edu/ark:/67531/met...,Web Archiving for all: Building WebRecorder.io,2015-04-28,"Kreymer, Ilya",digital preservation; web archiving; web crawl,Presentation for the 2015 International Intern...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,WebRecorder.io\nBuilding a new archiving servi...,*WebRecorder.io: Building a New Archiving Serv...


In [2]:
# Drop rows with null cleaned_text and convert to string
df = df[df['cleaned_text'].notnull()].copy()
df['cleaned_text'] = df['cleaned_text'].astype(str)

In [ ]:
from tqdm import tqdm
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")

# Ensure cleaned_text column has valid strings
df = df[df['cleaned_text'].notnull()].copy()
df['cleaned_text'] = df['cleaned_text'].astype(str)

# --- Make sure we have a document ID column ---
if 'doc_id' not in df.columns:
    df['doc_id'] = df.index.astype(str)

# Chunk settings
CHUNK_SIZE = 450
OVERLAP = 50

def chunk_by_tokens(text, tokenizer, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        start += chunk_size - overlap
    return chunks

# Chunk each row's cleaned_text
chunked_rows = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Token chunking"):
    text_chunks = chunk_by_tokens(row['cleaned_text'], tokenizer)
    for i, chunk in enumerate(text_chunks):
        new_row = row.to_dict()
        new_row['cleaned_text'] = chunk
        new_row['chunk_id'] = i          # position within the document
        chunked_rows.append(new_row)

# Final DataFrame: contains doc_id, chunk_id, cleaned_text
df = pd.DataFrame(chunked_rows).reset_index(drop=True)


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Token chunking: 100%|██████████| 575/575 [00:03<00:00, 153.65it/s]


In [4]:
df.head()

,ark_url,title,date,creator,subject,description,item_type,source_url,full_text,cleaned_text,doc_id,chunk_id
0,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",- - - # * * the end of term archive : collabor...,0,0
1,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",", and others utilize tools like archive - it. ...",0,1
2,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",- - # # * * 5. funding and development * * * *...,0,2
3,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",cocciolo _ peet _ eot. pdf ] ( http : / / www....,0,3
4,https://digital.library.unt.edu/ark:/67531/met...,The End of Term Archive: Collaboratively Prese...,2018-11-13,"Phillips, Mark Edward",digital libraries; government documents; web a...,Presentation the 2018 International Internet P...,Presentation,https://digital.library.unt.edu/ark:/67531/met...,"Mark Phillips, UNT Libraries \nAbbie Grotke, L...",change by percentage was observed. `. gov ` an...,0,4


In [ ]:
from sentence_transformers import SentenceTransformer
import pickle

# Load the embedding model
model = SentenceTransformer('BAAI/bge-base-en-v1.5')


df['combined_text'] = (
    "This item is titled '" + df['title'].fillna('') + 
    "' and was created by " + df['creator'].fillna('Unknown') + ". " +
    "It is about " + df['subject'].fillna('various topics') + ". " +
    "Description: " + df['description'].fillna('No description provided.') + " " +
    "Main content: " + df['cleaned_text'].fillna('') + " " +
    "It is classified as " + df['item_type'].fillna('unknown type') +
    ", dated " + df['date'].fillna('unknown date') + ". " +
    "You can find it at " + df['ark_url'].fillna('') + 
    " (source: " + df['source_url'].fillna('') + ")."
)



# Create embeddings for the combined text
embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True)

with open('/kaggle/working/embeddings_v3.pkl', 'wb') as f:
    pickle.dump({
        'embeddings': embeddings,
        'combined_texts': df['combined_text'].tolist(),
        'doc_ids': df['doc_id'].tolist()
    }, f)



print("✅ Embeddings and all metadata saved successfully.")


2025-08-13 17:52:26.715328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755107546.986725      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755107547.062125      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/74 [00:00<?, ?it/s]

✅ Embeddings and all metadata saved successfully.


In [6]:
pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pickle
import numpy as np
import faiss
# Load pickle data
with open('/kaggle/working/embeddings_v3.pkl', 'rb') as f:
    data = pickle.load(f)

embeddings = np.array(data['embeddings']).astype('float32')

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [ ]:
def retrieve_top_k(query,k_chunks=30, k_docs=3, k_final=10):

    # 1. Encode query
    query_embedding = model.encode([query])

    # 2. Search top chunks
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k_chunks)

    # 3. Collect results with doc_id
    retrieved = []
    for idx, dist in zip(indices[0], distances[0]):
        retrieved.append({
            'doc_id': data['doc_ids'][idx],
            'combined_text': data['combined_texts'][idx],
            'score': float(dist)
        })

    # 4. Group by doc_id and take best score per doc
    from collections import defaultdict
    doc_scores = defaultdict(list)
    for r in retrieved:
        doc_scores[r['doc_id']].append(r)

    # Rank docs by their best chunk score (lower distance = better)
    ranked_docs = sorted(doc_scores.items(), key=lambda x: min(c['score'] for c in x[1]))[:k_docs]

    # 5. From these top docs, flatten all their chunks and re-sort globally
    selected_chunks = []
    for _, chunks in ranked_docs:
        # sort chunks within doc by score
        chunks_sorted = sorted(chunks, key=lambda c: c['score'])
        selected_chunks.extend(chunks_sorted)

    # 6. Take top k_final chunks overall
    final_results = sorted(selected_chunks, key=lambda c: c['score'])[:k_final]
    return final_results


In [ ]:
import google.generativeai as genai

# Configure your Gemini API key
genai.configure(api_key="add-your-api-key-here")

MODEL_NAME = "gemini-2.5-flash-lite"  # Gemini 2.5 Flash-Lite

AiModel = genai.GenerativeModel(MODEL_NAME)

def generate_response(query, context_docs):
    context_parts = []
    for doc in context_docs:
        context_parts.append(
            f"{doc['combined_text']}\n----"
        )
    context = "\n".join(context_parts)

    prompt = f"""You are an expert digital preservation and web archiving assistant for the International Internet Preservation Consortium (IIPC). Your role is to provide comprehensive, accurate answers using ONLY the IIPC conference materials, presentations, and research papers provided below.

CONTEXT UNDERSTANDING:
Each document contains structured metadata including Title, Creator, Date, Subject, Description, Item Type, and extracted content from IIPC conferences spanning multiple years. These materials cover cutting-edge research, best practices, tools, and methodologies in web archiving and digital preservation.

RESPONSE REQUIREMENTS:

1. ACCURACY & SOURCE FIDELITY:
   - Base your answers STRICTLY on the provided context
   - Never add information from outside sources or general knowledge
   - If information is insufficient, clearly state "Based on the available IIPC materials, I don't have enough information to fully answer this question"

2. SOURCE ATTRIBUTION:
   - When referencing specific information, mention the source document title and author when available
   - For presentations, include the year of the conference when mentioned
   - Example: "According to [Author Name]'s presentation '[Title]' from the [Year] IIPC conference..."

3. COMPREHENSIVE COVERAGE:
   - Synthesize information from multiple relevant documents when applicable
   - Provide context about the evolution of topics across different conference years
   - Include both theoretical concepts and practical implementations mentioned

4. TECHNICAL ACCURACY:
   - Use precise terminology from web archiving and digital preservation fields
   - Explain technical concepts clearly while maintaining accuracy
   - Reference specific tools, standards (like WARC), and methodologies mentioned in the materials

5. RESPONSE STRUCTURE:
   - Start with a direct answer to the main question
   - Provide supporting details and examples from the materials
   - When relevant, mention different perspectives or approaches from various presenters
   - Conclude with practical implications or current relevance if discussed in the materials

6. FORMATTING:
   - Use clear, professional language appropriate for researchers and practitioners
   - Structure longer responses with clear paragraphs
   - Use simple bullet points (with dashes) only when listing distinct items or steps
   - Avoid markdown formatting symbols

7. TEMPORAL CONTEXT:
   - When discussing developments or trends, reference the timeframe based on conference dates
   - Highlight how approaches or technologies have evolved according to the materials
   - Note any historical context provided in the presentations

Context from IIPC Conference Materials:
{context}

User Question: {query}

Response:"""

    response = AiModel.generate_content(prompt)
    return response.text



In [12]:
query = "What legal issues affect web archiving?"
results = retrieve_top_k(query)
answer = generate_response(query, results)

print("\n🤖 Gemini Answer:\n", answer)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🤖 Gemini Answer:
 Based on the available IIPC materials, several legal issues affect web archiving:

Copyright is a significant legal issue, as mentioned in Alejandra Michel's presentation "The legal framework for web archiving: focussing on GDPR" from the 2019 IIPC conference. This presentation also notes that other legal issues include privacy law, data protection law, missions of national cultural heritage institutions, illegal contents, probative value of web archives, and the definition of the "national" web.

The General Data Protection Regulation (GDPR) in the European Union is also a key legal framework influencing web archiving. Alejandra Michel's presentation highlights that the GDPR applies to web archiving because it involves the processing of websites that may contain personal data. Personal data is defined as "any information relating to an identified or identifiable natural person," which in web archiving can include names, contact details, bibliographic data, photos, a